In [2]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import pandas as pd
import phate
import math
import random
import gc
import scprep
from datetime import datetime, time
from matplotlib.animation import ImageMagickWriter
import matplotlib.animation as animation
import zipfile
from urllib.request import urlopen
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import gaussian_kde
from scipy.stats import kde
from scipy.stats import binned_statistic
from scipy.stats import f_oneway
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['pdf.fonttype'] = 42
print(sns.__version__)
from anndata import AnnData
import scanpy as sc
from delve import *
import anndata as ad
from sklearn.preprocessing import MinMaxScaler
from kh import sketch
from sklearn.cluster import KMeans
import umap
print(sc.__version__)
today = datetime.now().strftime("%m%d%Y-%H%M")

0.11.2
1.9.1


In [3]:
#Read back in the subsampled adata file
adata_save_path = r'Z:\Garrett\4i_data\112723_RPE_31day_Etop\experiment\cell_data\standard_trimmed_noPSTAT5_adata_sub.h5ad'
standard_trimmed_noPSTAT5_adata_sub = anndata.read_h5ad(adata_save_path)

c:\Users\gases\.conda\envs\python_3_7\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
def laplacian_score_fs(adata = None,
                    k: int  = None,
                    n_jobs: int  = -1):

    X, feature_names, obs_names = parse_input(adata)
    W = construct_affinity(X = X, k = k, n_jobs = n_jobs)
    scores = laplacian_score(X = X, W = W)
    predicted_features = pd.DataFrame(scores, index = feature_names, columns = ['laplacian_score'])
    predicted_features = predicted_features.sort_values(by = 'laplacian_score', ascending = True)

    return predicted_features 

In [6]:
#l_score_fullest = laplacian_score_fs(adata_fullest_sub, k = 200)
l_score_standard = laplacian_score_fs(standard_trimmed_noPSTAT5_adata_sub, k = 100)
#l_score_normalized = laplacian_score_fs(normalized_trimmed_noPSTAT5_adata_sub, k = 200)

In [6]:
len(l_score_standard)

84

In [7]:
l_score_standard.index[:46]

Index(['nuc_area', 'R2_pp65_total_nuc_protein', 'R5_p38_total_nuc_protein',
       'R1_p16_total_nuc_protein', 'R4_PARP1_total_nuc_protein',
       'R3_p53_total_nuc_protein', 'R5_STAT5_total_nuc_protein',
       'R7_pp38_total_nuc_protein', 'R6_pSTAT3_total_nuc_protein',
       'R2_IL6_total_nuc_protein', 'R7_GATA4_total_nuc_protein',
       'R1_Jak2_total_nuc_protein', 'R6_p65_total_nuc_protein',
       'minor_axis_length', 'R0_Rb_total_nuc_protein',
       'R3_PLAUR_total_nuc_protein', 'R5_p38_nuc_mean', 'R0_pRb_nuc_mean',
       'major_axis_length', 'R4_IL8_total_nuc_protein',
       'R0_pRb_total_nuc_protein', 'R0_pRb_ratio', 'R1_p21_total_nuc_protein',
       'R0_DNA_total_nuc_protein', 'R6_pSTAT3_ring_mean', 'R2_IL6_ring_mean',
       'R6_p65_ring_mean', 'R1_p21_nuc_mean', 'R6_pSTAT3_nuc_mean',
       'R0_Rb_ratio', 'R5_p38_ring_mean', 'R6_p65_nuc_mean', 'R2_IL6_nuc_mean',
       'R1_p21_ratio', 'R0_Rb_nuc_mean', 'R7_GATA4_nuc_mean',
       'R1_p16_nuc_mean', 'R5_p38_ratio', 'R2

Code below taking inputs from Slingshot processing (Done in R) and any provided Clustering strategy.
Phate must be run and the PHATE coordinates saved using methods from the PHATE plotting notebook.
Produces a graph that plots specified clusters and overlays the Lineage information provided by Slingshot.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

# Plotting loop
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

for feature_name in var_names:
    # Calculate global min and max for the feature
    all_feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X.flatten()
    global_min = np.nanmin(all_feature_data)
    global_max = np.nanmax(all_feature_data)

    # Second pass: plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))  # Wide and short plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            z = np.polyfit(pseudotime_valid, feature_data_valid, 3)
            p = np.poly1d(z)
            pseudotime_sorted = np.sort(pseudotime_valid)
            plt_poly = p(pseudotime_sorted)

            # Plot data and polyfit
            scatter = axes[subplot_index].scatter(pseudotime_valid, feature_data_valid, alpha=0.5, 
                                                  c=[sample_id_to_color[id] for id in sample_ids_valid])
            axes[subplot_index].plot(pseudotime_sorted, plt_poly, color='red')
            axes[subplot_index].set_ylim(global_min, global_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

# Save the PowerPoint presentation
prs.save('feature_plots_loess.pptx')


Loess Curve Plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

# Plotting loop
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

for feature_name in var_names:
    # Calculate global min and max for the feature
    all_feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X.flatten()
    global_min = np.nanmin(all_feature_data)
    global_max = np.nanmax(all_feature_data)

    # Second pass: plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))  # Wide and short plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Generate LOESS curve
            loess_smoothed = lowess(feature_data_valid, pseudotime_valid, frac=0.3)
            pseudotime_sorted = loess_smoothed[:, 0]
            loess_values = loess_smoothed[:, 1]

            # Plot data and LOESS curve
            scatter = axes[subplot_index].scatter(pseudotime_valid, feature_data_valid, alpha=0.5, 
                                                  c=[sample_id_to_color[id] for id in sample_ids_valid])
            axes[subplot_index].plot(pseudotime_sorted, loess_values, color='red')
            axes[subplot_index].set_ylim(global_min, global_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

# Save the PowerPoint presentation
prs.save('feature_plots_loess.pptx')


Compare Loess Curves - Generate P value

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

# Plotting loop
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

# Lineages to compare
lineages_to_compare = [1, 3]

for feature_name in var_names:
    # Calculate global min and max for the feature
    all_feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X.flatten()
    global_min = np.nanmin(all_feature_data)
    global_max = np.nanmax(all_feature_data)

    # Data for statistical comparison
    lineage_data = {}

    # Second pass: plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))  # Wide and short plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in lineages_to_compare:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 2:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Generate LOESS curve
            loess_smoothed = lowess(feature_data_valid, pseudotime_valid, frac=0.3)
            pseudotime_sorted = loess_smoothed[:, 0]
            loess_values = loess_smoothed[:, 1]

            # Store data for comparison
            lineage_data[lineage_number] = {
                'pseudotime': pseudotime_sorted,
                'loess_values': loess_values
            }

            # Plot data and LOESS curve
            scatter = axes[subplot_index].scatter(pseudotime_valid, feature_data_valid, alpha=0.5, 
                                                  c=[sample_id_to_color[id] for id in sample_ids_valid])
            axes[subplot_index].plot(pseudotime_sorted, loess_values, color='red')
            axes[subplot_index].set_ylim(global_min, global_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

    # Statistical comparison of the two lineages
    if len(lineages_to_compare) == 2:
        lineage1 = lineages_to_compare[0]
        lineage2 = lineages_to_compare[1]
        
        if lineage1 in lineage_data and lineage2 in lineage_data:
            loess1 = lineage_data[lineage1]['loess_values']
            loess2 = lineage_data[lineage2]['loess_values']
            
            # Ensure same length for comparison
            min_len = min(len(loess1), len(loess2))
            loess1 = loess1[:min_len]
            loess2 = loess2[:min_len]
            
            # Perform t-test
            t_stat, p_value = ttest_ind(loess1, loess2)
            print(f"Feature: {feature_name}, Lineage {lineage1} vs Lineage {lineage2}, p-value: {p_value}")

# Save the PowerPoint presentation
prs.save('feature_plots_loess.pptx')


Feature: nuc_area, Lineage 1 vs Lineage 3, p-value: 9.587454874772373e-38
Feature: major_axis_length, Lineage 1 vs Lineage 3, p-value: 3.026432758148923e-14
Feature: minor_axis_length, Lineage 1 vs Lineage 3, p-value: 2.2003315051975713e-25
Feature: R0_DNA_nuc_mean, Lineage 1 vs Lineage 3, p-value: 0.0
Feature: R0_Rb_nuc_mean, Lineage 1 vs Lineage 3, p-value: 2.722685091838412e-27
Feature: R0_pRb_nuc_mean, Lineage 1 vs Lineage 3, p-value: 5.21188283198549e-15
Feature: R1_Jak2_nuc_mean, Lineage 1 vs Lineage 3, p-value: 0.0
Feature: R1_p16_nuc_mean, Lineage 1 vs Lineage 3, p-value: 3.593289436967946e-58
Feature: R1_p21_nuc_mean, Lineage 1 vs Lineage 3, p-value: 0.33671438926263286
Feature: R2_pp65_nuc_mean, Lineage 1 vs Lineage 3, p-value: 0.0
Feature: R2_IL6_nuc_mean, Lineage 1 vs Lineage 3, p-value: 4.6789408129290665e-163
Feature: R3_p53_nuc_mean, Lineage 1 vs Lineage 3, p-value: 0.0
Feature: R3_PLAUR_nuc_mean, Lineage 1 vs Lineage 3, p-value: 5.776255655233131e-100
Feature: R4_IL8_nu

Loess Curves - P value - Export as excel sheet

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.stats import ttest_ind
import pandas as pd

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# DataFrame to collect comparison results
comparison_results = []

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

# Plotting loop
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

# Lineages to compare
lineages = [1, 2, 3]

for feature_name in var_names:
    # Calculate global min and max for the feature
    all_feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X.flatten()
    global_min = np.nanmin(all_feature_data)
    global_max = np.nanmax(all_feature_data)

    # Data for statistical comparison
    lineage_data = {}

    # Second pass: plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))  # Adjusted for three plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in lineages:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Generate LOESS curve
            loess_smoothed = lowess(feature_data_valid, pseudotime_valid, frac=0.3)
            pseudotime_sorted = loess_smoothed[:, 0]
            loess_values = loess_smoothed[:, 1]

            # Store data for comparison
            lineage_data[lineage_number] = {
                'pseudotime': pseudotime_sorted,
                'loess_values': loess_values
            }

            # Plot data and LOESS curve
            scatter = axes[subplot_index].scatter(pseudotime_valid, feature_data_valid, alpha=0.5, 
                                                  c=[sample_id_to_color[id] for id in sample_ids_valid])
            axes[subplot_index].plot(pseudotime_sorted, loess_values, color='red')
            axes[subplot_index].set_ylim(global_min, global_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

    # Statistical comparison for the three pairs
    comparisons = [(1, 2), (2, 3), (1, 3)]
    
    comparison_result = {'Feature': feature_name}
    
    for lineage1, lineage2 in comparisons:
        if lineage1 in lineage_data and lineage2 in lineage_data:
            loess1 = lineage_data[lineage1]['loess_values']
            loess2 = lineage_data[lineage2]['loess_values']
            
            # Ensure same length for comparison
            min_len = min(len(loess1), len(loess2))
            loess1 = loess1[:min_len]
            loess2 = loess2[:min_len]
            
            # Perform t-test
            t_stat, p_value = ttest_ind(loess1, loess2)
            print(f"Feature: {feature_name}, Lineage {lineage1} vs Lineage {lineage2}, p-value: {p_value}")
            
            # Collect results for Excel
            comparison_result[f'Lineage {lineage1} vs Lineage {lineage2} p-value'] = p_value
    
    comparison_results.append(comparison_result)

# Save the PowerPoint presentation
prs.save('feature_plots_loess.pptx')

# Save the collected comparison results to an Excel file
comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_excel('lineage_comparison_results.xlsx', index=False)


Feature: nuc_area, Lineage 1 vs Lineage 2, p-value: 0.0
Feature: nuc_area, Lineage 2 vs Lineage 3, p-value: 0.0
Feature: nuc_area, Lineage 1 vs Lineage 3, p-value: 9.587454874772373e-38
Feature: major_axis_length, Lineage 1 vs Lineage 2, p-value: 0.0
Feature: major_axis_length, Lineage 2 vs Lineage 3, p-value: 0.0
Feature: major_axis_length, Lineage 1 vs Lineage 3, p-value: 3.026432758148923e-14
Feature: minor_axis_length, Lineage 1 vs Lineage 2, p-value: 0.0
Feature: minor_axis_length, Lineage 2 vs Lineage 3, p-value: 0.0
Feature: minor_axis_length, Lineage 1 vs Lineage 3, p-value: 2.2003315051975713e-25
Feature: R0_DNA_nuc_mean, Lineage 1 vs Lineage 2, p-value: 4.357640514449245e-67
Feature: R0_DNA_nuc_mean, Lineage 2 vs Lineage 3, p-value: 1.7820603848987537e-261
Feature: R0_DNA_nuc_mean, Lineage 1 vs Lineage 3, p-value: 0.0
Feature: R0_Rb_nuc_mean, Lineage 1 vs Lineage 2, p-value: 3.351583829695125e-70
Feature: R0_Rb_nuc_mean, Lineage 2 vs Lineage 3, p-value: 3.463391791333404e-14


KS test exporting to excel

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.stats import ks_2samp  # Use the Kolmogorov-Smirnov test
import pandas as pd

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# DataFrame to collect comparison results
comparison_results = []

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

# Plotting loop
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

# Lineages to compare
lineages = [1, 2, 3]

for feature_name in var_names:
    # Calculate global min and max for the feature
    all_feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X.flatten()
    global_min = np.nanmin(all_feature_data)
    global_max = np.nanmax(all_feature_data)

    # Data for statistical comparison
    lineage_data = {}

    # Second pass: plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))  # Adjusted for three plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in lineages:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Generate LOESS curve
            loess_smoothed = lowess(feature_data_valid, pseudotime_valid, frac=0.3)
            pseudotime_sorted = loess_smoothed[:, 0]
            loess_values = loess_smoothed[:, 1]

            # Store data for comparison
            lineage_data[lineage_number] = {
                'pseudotime': pseudotime_sorted,
                'loess_values': loess_values
            }

            # Plot data and LOESS curve
            scatter = axes[subplot_index].scatter(pseudotime_valid, feature_data_valid, alpha=0.5, 
                                                  c=[sample_id_to_color[id] for id in sample_ids_valid])
            axes[subplot_index].plot(pseudotime_sorted, loess_values, color='red')
            axes[subplot_index].set_ylim(global_min, global_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

    # Statistical comparison for the three pairs
    comparisons = [(1, 2), (2, 3), (1, 3)]
    
    comparison_result = {'Feature': feature_name}
    
    for lineage1, lineage2 in comparisons:
        if lineage1 in lineage_data and lineage2 in lineage_data:
            loess1 = lineage_data[lineage1]['loess_values']
            loess2 = lineage_data[lineage2]['loess_values']
            
            # Ensure same length for comparison
            min_len = min(len(loess1), len(loess2))
            loess1 = loess1[:min_len]
            loess2 = loess2[:min_len]
            
            # Perform Kolmogorov-Smirnov test
            ks_stat, p_value = ks_2samp(loess1, loess2)
            print(f"Feature: {feature_name}, Lineage {lineage1} vs Lineage {lineage2}, p-value: {p_value}")
            
            # Collect results for Excel
            comparison_result[f'Lineage {lineage1} vs Lineage {lineage2} p-value'] = p_value
    
    comparison_results.append(comparison_result)

# Save the PowerPoint presentation
prs.save('feature_plots_loess_ks.pptx')

# Save the collected comparison results to an Excel file
comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_excel('lineage_comparison_results_ks.xlsx', index=False)


Feature: nuc_area, Lineage 1 vs Lineage 2, p-value: 0.0
Feature: nuc_area, Lineage 2 vs Lineage 3, p-value: 0.0
Feature: nuc_area, Lineage 1 vs Lineage 3, p-value: 6.385689264e-315
Feature: major_axis_length, Lineage 1 vs Lineage 2, p-value: 0.0
Feature: major_axis_length, Lineage 2 vs Lineage 3, p-value: 0.0
Feature: major_axis_length, Lineage 1 vs Lineage 3, p-value: 2.052529334490752e-202
Feature: minor_axis_length, Lineage 1 vs Lineage 2, p-value: 0.0
Feature: minor_axis_length, Lineage 2 vs Lineage 3, p-value: 0.0
Feature: minor_axis_length, Lineage 1 vs Lineage 3, p-value: 2.633862180810416e-262
Feature: R0_DNA_nuc_mean, Lineage 1 vs Lineage 2, p-value: 5.22060737065959e-170
Feature: R0_DNA_nuc_mean, Lineage 2 vs Lineage 3, p-value: 0.0
Feature: R0_DNA_nuc_mean, Lineage 1 vs Lineage 3, p-value: 0.0
Feature: R0_Rb_nuc_mean, Lineage 1 vs Lineage 2, p-value: 4.399281090345645e-158
Feature: R0_Rb_nuc_mean, Lineage 2 vs Lineage 3, p-value: 8.412705010889853e-274
Feature: R0_Rb_nuc_mea

95% Confidence Interval from Loess curves - Works!

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

def bootstrap_loess(x, y, frac=0.3, n_boot=100, grid_size=100):
    grid = np.linspace(np.min(x), np.max(x), grid_size)
    loess_values_boot = []
    for _ in range(n_boot):
        sample_indices = np.random.choice(range(len(x)), size=len(x), replace=True)
        loess_smoothed = lowess(y[sample_indices], x[sample_indices], frac=frac)
        loess_interp = np.interp(grid, loess_smoothed[:, 0], loess_smoothed[:, 1])
        loess_values_boot.append(loess_interp)
    loess_values_boot = np.array(loess_values_boot)
    loess_mean = np.mean(loess_values_boot, axis=0)
    loess_ci_lower = np.percentile(loess_values_boot, 2.5, axis=0)
    loess_ci_upper = np.percentile(loess_values_boot, 97.5, axis=0)
    return grid, loess_mean, loess_ci_lower, loess_ci_upper

# Specify the feature to test
test_feature_name = 'R3_p53_total_nuc_protein'  # Replace with your specific feature name

# Plotting loop for the specified feature
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

# Check if the specified feature is in var_names
if test_feature_name in var_names:
    feature_name = test_feature_name
    
    # Calculate global min and max for the feature
    all_feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X.flatten()
    global_min = np.nanmin(all_feature_data)
    global_max = np.nanmax(all_feature_data)

    # Plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))  # Wide and short plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Generate LOESS curve and confidence intervals
            grid, loess_mean, loess_ci_lower, loess_ci_upper = bootstrap_loess(pseudotime_valid, feature_data_valid, frac=0.3)

            # Plot data and LOESS curve with confidence intervals
            scatter = axes[subplot_index].scatter(pseudotime_valid, feature_data_valid, alpha=0.5, 
                                                  c=[sample_id_to_color[id] for id in sample_ids_valid])
            axes[subplot_index].plot(grid, loess_mean, color='red')
            axes[subplot_index].fill_between(grid, loess_ci_lower, loess_ci_upper, color='blue', alpha=0.2)
            axes[subplot_index].set_ylim(global_min, global_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

    # Save the PowerPoint presentation
    prs.save('feature_plots_loess_R3_p53_total_nuc_protein.pptx')

else:
    print(f"Feature '{test_feature_name}' not found in var_names.")


95% CI for all features from Loess Curves

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

def bootstrap_loess(x, y, frac=0.3, n_boot=100, grid_size=100):
    grid = np.linspace(np.min(x), np.max(x), grid_size)
    loess_values_boot = []
    for _ in range(n_boot):
        sample_indices = np.random.choice(range(len(x)), size=len(x), replace=True)
        loess_smoothed = lowess(y[sample_indices], x[sample_indices], frac=frac)
        loess_interp = np.interp(grid, loess_smoothed[:, 0], loess_smoothed[:, 1])
        loess_values_boot.append(loess_interp)
    loess_values_boot = np.array(loess_values_boot)
    loess_mean = np.mean(loess_values_boot, axis=0)
    loess_ci_lower = np.percentile(loess_values_boot, 2.5, axis=0)
    loess_ci_upper = np.percentile(loess_values_boot, 97.5, axis=0)
    return grid, loess_mean, loess_ci_lower, loess_ci_upper

# Plotting loop for all features in var_names
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

for feature_name in var_names:
    # Calculate global min and max for the feature
    all_feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X.flatten()
    global_min = np.nanmin(all_feature_data)
    global_max = np.nanmax(all_feature_data)

    # Plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))  # Wide and short plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Generate LOESS curve and confidence intervals
            grid, loess_mean, loess_ci_lower, loess_ci_upper = bootstrap_loess(pseudotime_valid, feature_data_valid, frac=0.3)

            # Plot data and LOESS curve with confidence intervals
            scatter = axes[subplot_index].scatter(pseudotime_valid, feature_data_valid, alpha=0.5, 
                                                  c=[sample_id_to_color[id] for id in sample_ids_valid])
            axes[subplot_index].plot(grid, loess_mean, color='red')
            axes[subplot_index].fill_between(grid, loess_ci_lower, loess_ci_upper, color='blue', alpha=0.2)
            axes[subplot_index].set_ylim(global_min, global_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

# Save the PowerPoint presentation
prs.save('all_feature_plots_95CI_loess.pptx')


KeyboardInterrupt: 

Exception ignored in: 'statsmodels.nonparametric._smoothers_lowess.calculate_weights'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in sum
KeyboardInterrupt


95% CI No points plotted Single Feature

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

def bootstrap_loess(x, y, frac=0.3, n_boot=100, grid_size=100):
    grid = np.linspace(np.min(x), np.max(x), grid_size)
    loess_values_boot = []
    for _ in range(n_boot):
        sample_indices = np.random.choice(range(len(x)), size=len(x), replace=True)
        loess_smoothed = lowess(y[sample_indices], x[sample_indices], frac=frac)
        loess_interp = np.interp(grid, loess_smoothed[:, 0], loess_smoothed[:, 1])
        loess_values_boot.append(loess_interp)
    loess_values_boot = np.array(loess_values_boot)
    loess_mean = np.mean(loess_values_boot, axis=0)
    loess_ci_lower = np.percentile(loess_values_boot, 2.5, axis=0)
    loess_ci_upper = np.percentile(loess_values_boot, 97.5, axis=0)
    return grid, loess_mean, loess_ci_lower, loess_ci_upper

# Specify the feature to test
test_feature_name = 'R3_p53_total_nuc_protein'  # Replace with your specific feature name

# Plotting loop for the specified feature
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

# Check if the specified feature is in var_names
if test_feature_name in var_names:
    feature_name = test_feature_name
    
    # Calculate overall min and max for the confidence intervals
    all_ci_lower = []
    all_ci_upper = []

    # Plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))  # Wide and short plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Generate LOESS curve and confidence intervals
            grid, loess_mean, loess_ci_lower, loess_ci_upper = bootstrap_loess(pseudotime_valid, feature_data_valid, frac=0.3)

            all_ci_lower.append(loess_ci_lower)
            all_ci_upper.append(loess_ci_upper)

    # Compute global min and max of confidence intervals
    global_ci_lower_min = (np.nanmin(np.concatenate(all_ci_lower))*1.1)
    global_ci_upper_max = (np.nanmax(np.concatenate(all_ci_upper))*1.1)

    subplot_index = 0
    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Generate LOESS curve and confidence intervals
            grid, loess_mean, loess_ci_lower, loess_ci_upper = bootstrap_loess(pseudotime_valid, feature_data_valid, frac=0.3)

            # Plot data and LOESS curve with confidence intervals
            scatter = axes[subplot_index].plot(grid, loess_mean, color='red')
            axes[subplot_index].fill_between(grid, loess_ci_lower, loess_ci_upper, color='blue', alpha=0.2)
            axes[subplot_index].set_ylim(global_ci_lower_min, global_ci_upper_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

    # Save the PowerPoint presentation
    prs.save('feature_plots_loess_R3_p53_total_nuc_protein.pptx')

else:
    print(f"Feature '{test_feature_name}' not found in var_names.")


95% CI No Points single plotted feature on one graph

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

def bootstrap_loess(x, y, frac=0.3, n_boot=100, grid_size=100):
    grid = np.linspace(np.min(x), np.max(x), grid_size)
    loess_values_boot = []
    for _ in range(n_boot):
        sample_indices = np.random.choice(range(len(x)), size=len(x), replace=True)
        loess_smoothed = lowess(y[sample_indices], x[sample_indices], frac=frac)
        loess_interp = np.interp(grid, loess_smoothed[:, 0], loess_smoothed[:, 1])
        loess_values_boot.append(loess_interp)
    loess_values_boot = np.array(loess_values_boot)
    loess_mean = np.mean(loess_values_boot, axis=0)
    loess_ci_lower = np.percentile(loess_values_boot, 2.5, axis=0)
    loess_ci_upper = np.percentile(loess_values_boot, 97.5, axis=0)
    return grid, loess_mean, loess_ci_lower, loess_ci_upper

# Specify the feature to test
test_feature_name = 'R1_p21_total_nuc_protein'  # Replace with your specific feature name

# Plotting loop for the specified feature
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

# Check if the specified feature is in var_names
if test_feature_name in var_names:
    feature_name = test_feature_name
    
    # Calculate overall min and max for the confidence intervals
    all_ci_lower = []
    all_ci_upper = []

    # Create a single plot for all lineages
    fig, ax = plt.subplots(figsize=(10, 6))

    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Normalize pseudotime to [0, 1]
            pseudotime_min = np.min(pseudotime_valid)
            pseudotime_max = np.max(pseudotime_valid)
            pseudotime_normalized = (pseudotime_valid - pseudotime_min) / (pseudotime_max - pseudotime_min)

            # Generate LOESS curve and confidence intervals
            grid, loess_mean, loess_ci_lower, loess_ci_upper = bootstrap_loess(pseudotime_normalized, feature_data_valid, frac=0.3)

            all_ci_lower.append(loess_ci_lower)
            all_ci_upper.append(loess_ci_upper)

            # Plot data and LOESS curve with confidence intervals
            ax.plot(grid, loess_mean, label=f'Lineage {lineage_number}')
            ax.fill_between(grid, loess_ci_lower, loess_ci_upper, alpha=0.2)

    # Compute global min and max of confidence intervals
    global_ci_lower_min = np.nanmin(np.concatenate(all_ci_lower))
    global_ci_upper_max = np.nanmax(np.concatenate(all_ci_upper))

    ax.set_ylim(global_ci_lower_min, global_ci_upper_max)
    ax.set_xlabel('Normalized Pseudotime')
    ax.set_ylabel(feature_name)
    ax.legend(title="Lineages")

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

    # Save the PowerPoint presentation
    prs.save(r'featureplots\feature_plots_loess_R1_p21_total_nuc_protein.pptx')

else:
    print(f"Feature '{test_feature_name}' not found in var_names.")


95% CI No points One Graph Plotted all features

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

def bootstrap_loess(x, y, frac=0.3, n_boot=100, grid_size=100):
    grid = np.linspace(np.min(x), np.max(x), grid_size)
    loess_values_boot = []
    for _ in range(n_boot):
        sample_indices = np.random.choice(range(len(x)), size=len(x), replace=True)
        loess_smoothed = lowess(y[sample_indices], x[sample_indices], frac=frac)
        loess_interp = np.interp(grid, loess_smoothed[:, 0], loess_smoothed[:, 1])
        loess_values_boot.append(loess_interp)
    loess_values_boot = np.array(loess_values_boot)
    loess_mean = np.mean(loess_values_boot, axis=0)
    loess_ci_lower = np.percentile(loess_values_boot, 2.5, axis=0)
    loess_ci_upper = np.percentile(loess_values_boot, 97.5, axis=0)
    return grid, loess_mean, loess_ci_lower, loess_ci_upper

# Iterate over all features in var_names
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

for feature_name in var_names:
    # Calculate overall min and max for the confidence intervals
    all_ci_lower = []
    all_ci_upper = []

    # Create a single plot for all lineages
    fig, ax = plt.subplots(figsize=(10, 6))

    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Normalize pseudotime to [0, 1]
            pseudotime_min = np.min(pseudotime_valid)
            pseudotime_max = np.max(pseudotime_valid)
            pseudotime_normalized = (pseudotime_valid - pseudotime_min) / (pseudotime_max - pseudotime_min)

            # Generate LOESS curve and confidence intervals
            grid, loess_mean, loess_ci_lower, loess_ci_upper = bootstrap_loess(pseudotime_normalized, feature_data_valid, frac=0.3)

            all_ci_lower.append(loess_ci_lower)
            all_ci_upper.append(loess_ci_upper)

            # Plot data and LOESS curve with confidence intervals
            ax.plot(grid, loess_mean, label=f'Lineage {lineage_number}')
            ax.fill_between(grid, loess_ci_lower, loess_ci_upper, alpha=0.2)

    # Compute global min and max of confidence intervals
    global_ci_lower_min = np.nanmin(np.concatenate(all_ci_lower))
    global_ci_upper_max = np.nanmax(np.concatenate(all_ci_upper))

    ax.set_ylim(global_ci_lower_min, global_ci_upper_max)
    ax.set_xlabel('Normalized Pseudotime')
    ax.set_ylabel(feature_name)
    ax.legend(title="Lineages")

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

# Save the PowerPoint presentation after all slides are added
prs.save('featureplots/all_features_plots_95CI_loess.pptx')


95% CI No points Individual Graphs Plotted all features - This is the code that I actually used to generate the plots for the figures

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io
from statsmodels.nonparametric.smoothers_lowess import lowess

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Define colors for each lineage in hexadecimal
lineage_colors = {
    1: "#1f77b4",  # Blue
    2: "#ff7f0e",  # Orange
    3: "#ff0000"   # Red
    # Add more lineages and colors if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(3))  # Adjusted width to 3 inches

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(4), Inches(1), width=Inches(2.5))

def bootstrap_loess(x, y, frac=0.3, n_boot=100, grid_size=100):
    grid = np.linspace(np.min(x), np.max(x), grid_size)
    loess_values_boot = []
    for _ in range(n_boot):
        sample_indices = np.random.choice(range(len(x)), size=len(x), replace=True)
        loess_smoothed = lowess(y[sample_indices], x[sample_indices], frac=frac)
        loess_interp = np.interp(grid, loess_smoothed[:, 0], loess_smoothed[:, 1])
        loess_values_boot.append(loess_interp)
    loess_values_boot = np.array(loess_values_boot)
    loess_mean = np.mean(loess_values_boot, axis=0)
    loess_ci_lower = np.percentile(loess_values_boot, 2.5, axis=0)
    loess_ci_upper = np.percentile(loess_values_boot, 97.5, axis=0)
    return grid, loess_mean, loess_ci_lower, loess_ci_upper

# Iterate over all features in var_names
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

for feature_name in var_names:
    global_ci_lower_min = float('inf')
    global_ci_upper_max = float('-inf')
    
    # First pass to determine global y-axis limits for the feature
    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Normalize pseudotime to [0, 1]
            pseudotime_min = np.min(pseudotime_valid)
            pseudotime_max = np.max(pseudotime_valid)
            pseudotime_normalized = (pseudotime_valid - pseudotime_min) / (pseudotime_max - pseudotime_min)

            # Generate LOESS curve and confidence intervals
            grid, loess_mean, loess_ci_lower, loess_ci_upper = bootstrap_loess(pseudotime_normalized, feature_data_valid, frac=0.3)

            # Compute global min and max of confidence intervals for the current lineage
            global_ci_lower_min = min(global_ci_lower_min, np.nanmin(loess_ci_lower))
            global_ci_upper_max = max(global_ci_upper_max, np.nanmax(loess_ci_upper))

    # Second pass to create plots with consistent y-axis limits
    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            # Normalize pseudotime to [0, 1]
            pseudotime_min = np.min(pseudotime_valid)
            pseudotime_max = np.max(pseudotime_valid)
            pseudotime_normalized = (pseudotime_valid - pseudotime_min) / (pseudotime_max - pseudotime_min)

            # Generate LOESS curve and confidence intervals
            grid, loess_mean, loess_ci_lower, loess_ci_upper = bootstrap_loess(pseudotime_normalized, feature_data_valid, frac=0.3)

            # Create a plot for the current lineage
            fig, ax = plt.subplots(figsize=(3.33, 6))  # Adjusted figsize to make the plot narrower
            ax.plot(grid, loess_mean, color=lineage_colors[lineage_number], label=f'Lineage {lineage_number}')
            ax.fill_between(grid, loess_ci_lower, loess_ci_upper, color=lineage_colors[lineage_number], alpha=0.2)

            # Set global min and max of confidence intervals for y-axis
            ax.set_ylim(global_ci_lower_min, global_ci_upper_max)

            ax.set_xlabel('Normalized Pseudotime')
            ax.set_ylabel(feature_name)
            ax.legend(title="Lineages")

            plt.tight_layout()
            
            # Save the plot to a BytesIO object
            img_data = io.BytesIO()
            plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
            img_data.seek(0)
            plt.close()

            # Create and save the legend
            fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
            ax_legend.axis('off')
            legend_handles = [plt.Line2D([0], [0], marker='o', color=lineage_colors[id], linestyle='', label=f'Lineage {id}') for id in pseudotime_cutoffs.keys()]
            ax_legend.legend(handles=legend_handles, title="Lineages", loc='center', fontsize='small', frameon=False)
            legend_img_data = io.BytesIO()
            plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
            legend_img_data.seek(0)
            plt.close()

            # Add the plot and the legend to the PowerPoint slide
            add_slide_with_plot(prs, img_data, f'{feature_name} expression - Lineage {lineage_number}', legend_img_data)

# Save the PowerPoint presentation after all slides are added
prs.save('featureplots/all_features_individual_plots_95CI_loess.pptx')


Not really related to the other code - Using this section to cut out the last 10% of each lineage for plotting

In [11]:
import os
import pandas as pd
import numpy as np

# Extract the relevant pseudotime columns from .obs
pseudotime_df = standard_trimmed_noPSTAT5_adata_sub.obs[['Pseudotime_Lineage1', 'Pseudotime_Lineage2', 'Pseudotime_Lineage3']].copy()

# Apply the cutoff for Lineage 2 and filter values accordingly
pseudotime_df['Pseudotime_Lineage2'] = pseudotime_df['Pseudotime_Lineage2'].apply(lambda x: x if x <= 0.02 else None)

# Calculate the 90th percentile for each lineage
percentile_90 = pseudotime_df.quantile(0.90)

# Function to filter and save data for a given lineage
def filter_and_save_lineage(lineage_number, cutoff, output_folder):
    pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
    if pseudotime_col in pseudotime_df.columns:
        mask = pseudotime_df[pseudotime_col] >= cutoff
        filtered_data = standard_trimmed_noPSTAT5_adata_sub[mask, :].to_df()
        csv_filename = os.path.join(output_folder, f'filtered_cells_last_10_percent_lineage_{lineage_number}.csv')
        filtered_data.to_csv(csv_filename, index=False)
        print(f'Saved {csv_filename}')

# Define the output folder
output_folder = r'C:\Users\gases\Desktop\Lineage_Filtering'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Filter and save data for each lineage
for lineage_number in range(1, 4):
    cutoff = percentile_90[f'Pseudotime_Lineage{lineage_number}']
    filter_and_save_lineage(lineage_number, cutoff, output_folder)
